In [ ]:
run_date = ''
output_bucket = ''

### Get tables from BigQuery

In [ ]:
from google.cloud import bigquery
def bigquery_table_to_df(project_id, dataset_id, table_id):
    client = bigquery.Client(project_id)
    
    table_ref = bigquery.DatasetReference(project_id, dataset_id).table(table_id)
    table = client.get_table(table_ref)

    df = client.list_rows(table).to_dataframe()
    
    return df

In [ ]:
df_div_reg = bigquery_table_to_df("analytics-ahs-owned-thd", "STG5", f'HDE_FALLOFF_EXCEL_DIVISION_REGION_{run_date}')
df_div_branch = bigquery_table_to_df("analytics-ahs-owned-thd", "STG5", f"HDE_FALLOFF_EXCEL_BOTTOM_5_BRANCH_PER_DIVISION_{run_date}")

In [ ]:
df_div_reg

In [ ]:
df_div_branch

### openpyxl testing: Create workbook and write data to it

In [ ]:
test_date_1 = '2021-01-01'
test_fw = 1
test_fw_begin = '1/1'
test_fw_end = '1/7'

header = ['Fall Off Report', 
          f'data refreshed: {test_date_1}', 
          f'Uses LMS BOW snapshot from {test_fw_begin}, beginning of FW{test_fw} (jobs that were scheduled to be installed between {test_fw_begin} & {test_fw_end})',
          f'HDIS Scorecard Data: FW{test_fw}',
          '',
          'by Region',
          '',
          '',
          ''
         ]

len(header)

In [ ]:
from google.cloud import storage

def upload_blob(source_file_name, project_id, bucket_name, destination_blob_name):
    """Uploads a file to the bucket."""

    storage_client = storage.Client(project_id)
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [ ]:
from openpyxl.workbook import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

rows = dataframe_to_rows(df_div_reg, index=False, header=False)

wb = Workbook()

ws = wb.active

header_length = len(header)

for i, header_val in enumerate(header):
    ws.cell(row = i+1, column = 2).value = header[i]

for r_idx, row in enumerate(rows, 1):
    for c_idx, value in enumerate(row, 1):
         ws.cell(row=r_idx + header_length, column=c_idx, value=value)

wb.save("../FallOffReport.xlsx")

In [ ]:
upload_blob(source_file_name = "../FallOffReport.xlsx", project_id = "analytics-ahs-owned-thd",
            bucket_name = output_bucket, destination_blob_name = "FallOffReport.xlsx")